# Fetch Data

Fetches all Anilist data needed for practicing data visualization and machine learning.

In [4]:
import csv
import json
import os
from datetime import datetime
from anilist_fetch import fetch as anilist

# setup
data_dir = os.path.abspath('data')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

## Fetch User Data

In [2]:
ANILIST_USER_ID = 247578
user = anilist.get_user(ANILIST_USER_ID)
print(user['user'])

# write user anime entries to CSV
with open(os.path.join(data_dir, f"user-{ANILIST_USER_ID}-entries.csv"), 'w+', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'status', 'score', 'progress', 'completedAt', 'media_id'])

    for media_list in user['lists']:
        for entry in media_list['entries']:
            data = [
                entry['id'],
                entry['status'],
                entry['score'],
                entry['progress'],
                None,
                entry['media']['id']
            ]
            completed_at = entry['completedAt']

            if completed_at['year'] and completed_at['month'] and completed_at['day']:
                data[4] = f"{completed_at['month']:02d}-{completed_at['day']:02d}-{completed_at['year']}"

            writer.writerow(data)

{'name': 'barrettotte', 'createdAt': 1552085204, 'statistics': {'anime': {'count': 605, 'meanScore': 68.93, 'standardDeviation': 15.9, 'minutesWatched': 198339, 'episodesWatched': 8119}}}


## Fetch All Anime Entries

In [5]:
start_date = datetime.strptime('19000101', '%Y%m%d').date()
end_date = datetime.today()
out_csv = f"anime-{start_date.strftime('%Y%m%d')}-{end_date.strftime('%Y%m%d')}.csv"

i = anilist.download_anime_range(start_date, end_date, os.path.join(data_dir, out_csv))
print(f"{i} entries downloaded")